In [1]:
import emnist
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
import methods as M
import sklearn
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.layers import Input, Lambda, Conv2D, MaxPooling2D, BatchNormalization, Dense, Flatten, Activation, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K
%matplotlib inline

In [2]:
train_images, train_labels, oneshot_images, oneshot_labels, classify_images, classify_labels = M.get_emnist(10, 1, True, False, 7)

======= Loading emnist data ... =======
Output shapes:  [(24000, 28, 28), (24000,), (16800, 28, 28), (16800,)]
Train labels:  [ 0  1  6 17 26 31 32 37 38 41]
Test labels:  [13 15 21 24 28 43 44]
======= Finished loading. =======


In [3]:
def emnist_nn_classifier(output):
    """
    Instanciate and compiles an autoencoder, returns both the autoencoder and just the encoder

    :param int or tuple input_size: shape of the input samples
    :param int code_size: dimension on which to project the original data
    :return: autoencoder, encoder
    """
    # YOUR CODE HERE
    encoder = keras.Sequential([
        keras.layers.Conv2D(32, (3,3), activation='ReLU'),
        keras.layers.MaxPool2D((2, 2)),
        keras.layers.Conv2D(16, (3,3), activation='ReLU'),
        keras.layers.MaxPool2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(32),
    ])
    
    decoder = keras.Sequential([
        keras.layers.Activation('ReLU'),
        keras.layers.Dense(output, activation = 'Softmax')
    ])
    
    inputs = keras.Input(shape=(28, 28, 1))
    model = keras.Model(inputs=inputs, outputs = decoder(encoder(inputs)))
    model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model, encoder

In [4]:
model, encoder = emnist_nn_classifier(10)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 sequential (Sequential)     (None, 32)                17776     
                                                                 
 sequential_1 (Sequential)   (None, 10)                330       
                                                                 
Total params: 18,106
Trainable params: 18,106
Non-trainable params: 0
_________________________________________________________________


In [5]:
def relabel(label):
    labelset = np.unique(label)
    kernel = np.zeros((np.max(labelset)+1,), dtype='int')
    temp = np.arange(len(labelset))
    kernel[labelset] = temp
    return kernel[label]

In [6]:
train_labels[:10]

array([26, 32,  1, 17, 32, 17, 26, 17, 32, 38], dtype=uint8)

In [7]:
y = relabel(train_labels)
y[:10]

array([4, 6, 1, 3, 6, 3, 4, 3, 6, 8])

In [8]:
model.fit(train_images, y, epochs=5)

Epoch 1/5
750/750 [==============================] - 20s 26ms/step - loss: 0.4657 - accuracy: 0.8556
Epoch 2/5
750/750 [==============================] - 19s 26ms/step - loss: 0.1924 - accuracy: 0.9415
Epoch 3/5
750/750 [==============================] - 19s 26ms/step - loss: 0.1486 - accuracy: 0.9546
Epoch 4/5
750/750 [==============================] - 19s 26ms/step - loss: 0.1261 - accuracy: 0.9612
Epoch 5/5
750/750 [==============================] - 19s 26ms/step - loss: 0.1080 - accuracy: 0.9659


In [9]:
oneshot_images.shape

(7, 28, 28)

In [10]:
oneshot_images_ = encoder(np.expand_dims(oneshot_images, axis=-1))
classify_images_ = encoder(np.expand_dims(classify_images, axis=-1))
nn = 1
neigh = KNeighborsClassifier(n_neighbors = nn)
neigh.fit(oneshot_images_, oneshot_labels)
pred = neigh.predict(classify_images_)
print("Accuracy: ", np.sum(pred == classify_labels)/len(classify_labels))

Accuracy:  0.6252605252188412


In [11]:
# random: 1/7 = 0.142